https://stackoverflow.com/questions/18308529/python-requests-package-handling-xml-response

In [162]:
from vocab_miner import sparql

import requests
import pandas as pd

import cx_Oracle
import configparser
from xml.etree import ElementTree

In [163]:
config = configparser.ConfigParser()
config.read('C:\workspace\config.ini')
oraprod = config['oraprod']

In [164]:

WFS_URL="http://geossdi.dmp.wa.gov.au/services/wfs"

In [165]:
get_feature = {"version":"1.1.0",
               "service": "wfs",
               "request": "GetFeature",
               "typeName": "mo:MinOccView",
               "maxFeatures" : 200,
               "outputFormat":"gml32",
               "startIndex": 0}

In [166]:
get_feature

{'maxFeatures': 200,
 'outputFormat': 'gml32',
 'request': 'GetFeature',
 'service': 'wfs',
 'startIndex': 0,
 'typeName': 'mo:MinOccView',
 'version': '1.1.0'}

In [167]:
ns = {'mo': 'http://xmlns.geoscience.gov.au/minoccml/1.0',
     'wfs': 'http://www.opengis.net/wfs/2.0'}

In [168]:
startIndex = get_feature["startIndex"]
commods = set()
while (startIndex < 120000):
    response=requests.get(WFS_URL,params=get_feature)
    xml  = response.text
    root = ElementTree.fromstring(xml)
    commods |= { commod for element in root.findall('wfs:member/mo:MinOccView/mo:commodity',ns) for commod in element.text.split(',')}
    startIndex = startIndex+ 200
    get_feature["startIndex"] = startIndex
    

In [169]:
len(commods)

249

In [170]:
wa_commodities = pd.read_excel('GS_MINEDEX_dbo_COMMODITIES.xlsx')

In [171]:
subset_commods = wa_commodities[wa_commodities["ABBREVIATION"].isin(commods) | wa_commodities["NAME"].isin(commods)]

In [172]:
subset_commods=subset_commods[["COMM_ID","NAME","ABBREVIATION"]]

In [173]:
remaining = commods - (set(subset_commods["ABBREVIATION"]) | set(subset_commods["NAME"]))

In [174]:
subset_commods

,COMM_ID,NAME,ABBREVIATION
0,1,Agate,Aga
1,2,Aggregate - coarse,Aggc
2,4,Aggregate - decorative,Aggd
4,6,Aggregate - marine,Aggm
5,7,Aggregate - undifferentiated,Agg
11,13,Aluminium,Al
13,15,Alunite,Alu
17,19,Andalusite,And
18,20,Anhydrite,Anh
20,22,Antimony,Sb


In [175]:
connection = cx_Oracle.connect(user=oraprod["user"],password=oraprod["password"],dsn=oraprod["dsn"])
c = connection.cursor()


In [176]:
def get_from_ga(commod):
    query = "select preflabel, uri from mgd.commodtypes where commodid = '{0}'"
    print(query.format(commod))
    c.execute(query.format(commod))
    result = c.fetchall()
    if len(result) > 0:
        print(result[0])
        return result[0]
    return None

In [177]:
def populate_labels(commod):
    ga = get_from_ga(commod)
    if ga is not None:
        return ga
    else:
        return sparql.get_exact(commod)

In [178]:
subset_commods[["PREFLABEL","URI"]]=subset_commods["ABBREVIATION"].apply(get_from_ga).apply(pd.Series)

select preflabel, uri from mgd.commodtypes where commodid = 'Aga'
('agate', 'http://resource.geosciml.org/classifier/cgi/commodity-code/agate')
select preflabel, uri from mgd.commodtypes where commodid = 'Aggc'
(None, None)
select preflabel, uri from mgd.commodtypes where commodid = 'Aggd'
(None, None)
select preflabel, uri from mgd.commodtypes where commodid = 'Aggm'
(None, None)
select preflabel, uri from mgd.commodtypes where commodid = 'Agg'
('aggregate', 'http://resource.geosciml.org/classifier/cgi/commodity-code/aggregate')
select preflabel, uri from mgd.commodtypes where commodid = 'Al'
('aluminium', 'http://resource.geosciml.org/classifier/cgi/commodity-code/aluminium')
select preflabel, uri from mgd.commodtypes where commodid = 'Alu'
('alunite', 'http://resource.geosciml.org/classifier/cgi/commodity-code/alunite')
select preflabel, uri from mgd.commodtypes where commodid = 'And'
('andalusite', 'http://resource.geosciml.org/classifier/cgi/commodity-code/andalusite')
select pref

In [180]:
subset_commods[["PREFLABEL","URI"]]=subset_commods["NAME"].apply(sparql.get_exact).apply(pd.Series)

ValueError: Columns must be same length as key

In [181]:
subset_commods

,COMM_ID,NAME,ABBREVIATION,PREFLABEL,URI
0,1,Agate,Aga,agate,http://resource.geosciml.org/classifier/cgi/co...
1,2,Aggregate - coarse,Aggc,None,None
2,4,Aggregate - decorative,Aggd,None,None
4,6,Aggregate - marine,Aggm,None,None
5,7,Aggregate - undifferentiated,Agg,aggregate,http://resource.geosciml.org/classifier/cgi/co...
11,13,Aluminium,Al,aluminium,http://resource.geosciml.org/classifier/cgi/co...
13,15,Alunite,Alu,alunite,http://resource.geosciml.org/classifier/cgi/co...
17,19,Andalusite,And,andalusite,http://resource.geosciml.org/classifier/cgi/co...
18,20,Anhydrite,Anh,anhydrite,http://resource.geosciml.org/classifier/cgi/co...
20,22,Antimony,Sb,antimony,http://resource.geosciml.org/classifier/cgi/co...
